# Machine Learning Template

After  niyo daanan yung steps dito, mags-save lahat ng processes na dinaanan niyo sa tested_iterations.csv.

Unfortunately, sa dulo niyo pa makikita kung may naulit kayong iteration based sa inputs niyo, so better if sa simula pa lang nagc-check na kayo ng past iterations from the csv.



### Don't forget to activate and deactivate your virtual environment! Pwede mag-crash laptop niyo because of this ehe

open cmd terminal sa VS Code

BEFORE STARTING:
`env\Scripts\activate.bat`


BEFORE CLOSING VS CODE:
`env\Scripts\deactivate.bat`

In [1]:
#imports 
import pandas as pd
import numpy as np
import json

import sklearn

from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression

from sklearn.metrics import r2_score
from sklearn.metrics import make_scorer

from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


In [2]:
x_train = pd.read_csv("./current iteration files/x_train.csv")
y_train = pd.read_csv("./current iteration files/y_train.csv")
x_test = pd.read_csv("./current iteration files/x_test.csv")
y_test = pd.read_csv("./current iteration files/y_test.csv")

Choose a base model

In [3]:
model_options = [DecisionTreeRegressor(), RandomForestRegressor(), SVR(), LinearRegression()]

model_string = ""
for i in range(len(model_options)):
    model_string = model_string + str(i) + " : " + str(model_options[i]) + " | "

valid_input = False
while valid_input == False:

    choice = input("Choose your base model. " + model_string + "Input the integer of your choice. ")

    try:
        choice = int(choice)
        if choice in range(len(model_options)):
            valid_input = True
    except:
        print("That is not a valid input. Choose again.")

base_model = model_options[choice]

#fit/train model
base_model.fit(x_train, y_train)

#make predictions 
predictions = base_model.predict(x_test)

#base evaluation
base_r2 = round(r2_score(y_test, predictions),2)
print("base_r2: {:.2f}".format(base_r2))


base_r2: -0.92


#### Hyperparameter tuning

Edit the grid variable for hyperparameter values you want to test.

You can also use RandomSearchCV for hyperparameter tuning

In [21]:
#Keep the grid variable sorted/alphabetized at each level
grid = {'max_depth':[4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150], 'criterion':['poisson', 'squared_error']}
grid = dict(sorted(grid.items()))


cvFold = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
gridSearch = GridSearchCV(estimator=base_model, param_grid=grid, n_jobs=-1,
	cv=cvFold, scoring=make_scorer(r2_score))
searchResults = gridSearch.fit(x_train, y_train)

# extract the best model and evaluate it
bestModel = searchResults.best_estimator_
best_r2 = round(bestModel.score(x_test, y_test))
print("best_r2: {:.2f}".format(best_r2))

best_r2: 1.00


In [22]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

In [23]:
ml_steps = {"base_model":str(base_model), "base_r2":base_r2, "hyperparameter tuning":str(gridSearch),"best_model":str(bestModel), "best_r2":best_r2}

ml = pd.json_normalize(ml_steps)


with open("./current iteration files/preprocessing.json", "r") as outfile: 
    preprocessing_steps = json.load(outfile, )

preprocessing = pd.json_normalize(preprocessing_steps)

all_process = pd.concat([preprocessing, ml], axis=1)
pd.set_option("display.max_colwidth", None)
print(all_process["hyperparameter tuning"][0])

GridSearchCV(cv=RepeatedKFold(n_repeats=3, n_splits=10, random_state=1),
             estimator=DecisionTreeRegressor(), n_jobs=-1,
             param_grid={'criterion': ['poisson', 'squared_error']},
             scoring=make_scorer(r2_score))


Update iterations

In [24]:
try:
    iterations = pd.read_csv("./tested_iterations.csv", dtype=str)
    new_iterations = pd.concat([iterations,all_process.astype(str)], ignore_index=True, axis=0)
    if new_iterations.duplicated().sum() != 0:
        print("Current steps are duplicate of previous iteration. Current iteration will not be saved in the file.")
    else:
        new_iterations.to_csv("./tested_iterations.csv", index=False)

except:
    all_process.to_csv("./tested_iterations.csv", index=False)

